In [80]:
import torch
from torch import nn
import numpy as np

In [82]:
import random
class myConv1d():
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int = 1) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        # init parameters
        parameters = []
        for i_out in range(out_channels):
            out_channel = []
            for i_in in range(in_channels):
                kernel = np.random.rand(kernel_size)
                out_channel.append(kernel)
            parameters.append(out_channel)
        self.parameters = torch.tensor(parameters)

        # init bias
        bias = []
        for i_out in range(out_channels):
            bias.append(random.random())
            # bias.append(0.1)
        self.bias = torch.tensor(bias)

    def __call__(self, input: torch.Tensor) -> torch.Tensor:
        calculated = []
        for i_out in range(self.out_channels):
            sum = [0.] * (input.size(1) - self.kernel_size + 1)
            for i_in in range(self.in_channels):
                for i_w in range(input.size(1) - self.kernel_size + 1):
                    # print(i_out, i_in, i_w)
                    sum[i_w] += torch.dot(self.parameters[i_out][i_in], input[i_in][i_w:i_w + self.kernel_size])
            sum = [x + self.bias[i_out] for x in sum]
            calculated.append(sum)
        return torch.tensor(calculated)

In [89]:
IN_CHANNEL = 2
OUT_CHANNEL = 2
KERNEL_SIZE = 3
HEIGHT = 3
WIDTH = 3
LENGTH = 4
t = torch.arange(IN_CHANNEL * LENGTH, dtype=torch.float)
t.resize_(IN_CHANNEL, LENGTH)
t

tensor([[0., 1., 2., 3.],
        [4., 5., 6., 7.]])

In [90]:
d1 = nn.Conv1d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE, dtype=torch.float)
d2 = myConv1d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d2.parameters = d1._parameters['weight'].data
d2.bias = d1._parameters['bias'].data
d2(t), d1(t)

(tensor([[-0.6806, -0.9437],
         [ 1.4369,  1.9290]]),
 tensor([[-0.6806, -0.9437],
         [ 1.4369,  1.9290]], grad_fn=<SqueezeBackward1>))

In [85]:
import random
class myConv2d():
    def __init__(self, in_channels: int, out_channels: int, kernel_size: tuple[int, int], stride: int = 1) -> None:
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        # init parameters
        parameters = []
        for i_out in range(out_channels):
            out_channel = []
            for i_in in range(in_channels):
                kernel = np.random.rand(*kernel_size)
                out_channel.append(kernel)
            parameters.append(out_channel)
        self.parameters = torch.tensor(parameters)

        # init bias
        bias = []
        for i_out in range(out_channels):
            bias.append(random.random())
            # bias.append(0.1)
        self.bias = torch.tensor(bias)

    def __call__(self, input: torch.Tensor) -> torch.Tensor:
        calculated = []
        for i_out in range(self.out_channels):
            sum = np.array([input.size(1) - self.kernel_size[0] + 1, input.size(2) - self.kernel_size[1] + 1])
            for i_in in range(self.in_channels):
                for i_w in range(input.size(1) - self.kernel_size[0] + 1):
                    for i_h in range(input.size(2) - self.kernel_size[1] + 1):
                        sum[i_w] += torch.sum(self.parameters[i_out][i_in] * input[i_in][i_w:i_w + self.kernel_size[0]][i_h:i_h + self.kernel_size[1]])
            sum = [x + self.bias[i_out] for x in sum]
            calculated.append(sum)
        return torch.tensor(calculated)

In [86]:
IN_CHANNEL = 2
OUT_CHANNEL = 2
KERNEL_SIZE = (3, 3)
HEIGHT = 3
WIDTH = 3
LENGTH = 4
t1 = torch.arange(IN_CHANNEL * LENGTH * HEIGHT, dtype=torch.float)
t1.resize_(IN_CHANNEL, LENGTH, HEIGHT)
t1

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.],
         [ 9., 10., 11.]],

        [[12., 13., 14.],
         [15., 16., 17.],
         [18., 19., 20.],
         [21., 22., 23.]]])

In [87]:
d3 = nn.Conv2d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d4 = myConv2d(IN_CHANNEL, OUT_CHANNEL, KERNEL_SIZE)
d4.parameters = d3._parameters['weight'].data
d4.bias = d3._parameters['bias'].data
d3(t1), d4(t1)

(tensor([[[ 9.1929],
          [10.9184]],
 
         [[-2.8325],
          [-3.4957]]], grad_fn=<SqueezeBackward1>),
 tensor([[ 9.9833, 10.9833],
         [-1.0271, -3.0271]]))

In [88]:
d4.bias

tensor([-0.0167, -0.0271])